# Python Code

In [121]:
import gurobipy as grb
import pandas as pd
import numpy as np
import datetime
from datetime import time
from datetime import timedelta
import math
import random

# for the room capacity file i remove the first record because it is same as the 7th record and will make some error in next step
capacity=pd.read_excel("Marshall_Room_Capacity_Chart.xlsx")
capacity1=capacity
course_enrollment=pd.read_excel("Marshall_Course_Enrollment_1516_1617.xlsx") # for student prediction


In [122]:
course_enrollment.head()

,Course,Course Prefix,Course Suffix,Department,First Begin Time,First Days,First End Time,First Instructor,First Instructor UID,First Room,...,Second Begin Time,Second Days,Second End Time,Second Instructor,Second Instructor UID,Second Room,Section,Session,Term,Title
0,ACCT-370,ACCT,370,ACCT,10:00:00,F,11:50:00,"Hopkins, Merle, W",3.783354e+09,SLH200,...,NaN,NaN,NaN,NaN,NaN,NaN,14029,1,20153,External Financial Reporting Issues
1,ACCT-370,ACCT,370,ACCT,08:00:00,MW,09:50:00,"Hopkins, Merle, W",3.783354e+09,ACC303,...,NaN,NaN,NaN,NaN,NaN,NaN,14025,1,20153,External Financial Reporting Issues
2,ACCT-370,ACCT,370,ACCT,10:00:00,MW,11:50:00,"Hopkins, Merle, W",3.783354e+09,ACC303,...,NaN,NaN,NaN,NaN,NaN,NaN,14026,1,20153,External Financial Reporting Issues
3,ACCT-370,ACCT,370,ACCT,12:00:00,MW,13:50:00,"Hopkins, Merle, W",3.783354e+09,ACC303,...,NaN,NaN,NaN,NaN,NaN,NaN,14027,1,20153,External Financial Reporting Issues
4,ACCT-371,ACCT,371,ACCT,10:00:00,F,11:50:00,NaN,NaN,SLH200,...,NaN,NaN,NaN,NaN,NaN,NaN,14044,1,20153,Introduction to Accounting Systems


## Extra data from original dataset

In [123]:
# choose 20153 term as sample
data=course_enrollment.loc[course_enrollment.loc[:,"Term"]==20153,("Course","Section","First Days",
                                                                   "First Begin Time","First End Time",
                                                                   "First Room","First Instructor",
                                                                   "Reg Count","Seats")]
data.columns=["Course","Section","FirstDays","FirstBeginTime",
              "FirstEndTime","FirstRoom","FirstInstructor","regcount","Seats"]
# we only use those M/T/W/H/F/MW/TH
data=data[(data.FirstDays=="M")|(data.FirstDays=="T")|(data.FirstDays=="W")|(data.FirstDays=="H")|(data.FirstDays=="F")|(data.FirstDays=="MW")|(data.FirstDays=="TH")]
data=data[(data.FirstRoom!="ONLINE")&(data.FirstRoom!="DEN@Viterbi")]
# remove the record that have nan in first begintime
data=data[(data.FirstBeginTime==data.FirstBeginTime)|(data.FirstEndTime==data.FirstEndTime)]
# remove the record that have nan in professor
data=data[data.FirstInstructor==data.FirstInstructor]


In [124]:
# calculate the timelength for each section
timelength=[]
for i in data.index:
    time1=data.loc[i,"FirstEndTime"]
    time2=data.loc[i,"FirstBeginTime"]
    minutediff=((time1.hour*60+time1.minute)-(time2.hour*60+time2.minute)+10)
    halfhour=minutediff/30
    if minutediff%30>0:
        halfhour+=1
    timelength.append(halfhour)

In [125]:
# get the pattern of each section, if section only have 
# one session per week so pattern=1, two sessions per week so pattern=2
pattern=[]
for i in data.index:
    day=data.loc[i,"FirstDays"]
    if (day=="M") or (day=="T") or (day=="W") or (day=="H") or (day=="F"):
        #print day
        pattern.append(1)
    elif (day=="MW") or (day=="TH"):
        #print day
        pattern.append(2)

In [126]:
# students' registeration prediction
# we predict the reg count for each section based on historic data 
student={}
for i in data["Section"]:
    sum1=sum(course_enrollment.loc[course_enrollment.loc[:,"Section"]==i,"Reg Count"])
    len1=len(course_enrollment.loc[course_enrollment.loc[:,"Section"]==i,"Reg Count"])
    student[i]=round(sum1/len1)

In [127]:
# build section_info table
section_info=pd.DataFrame({"course":data["Course"],"section":data["Section"],"pattern":pattern,
                           "timelength":timelength,"FirstInstructor":data["FirstInstructor"],
                           "Reg Count":student.values()})
# we only use those timelength less than or equal to 4 hours
section_info=section_info[section_info.timelength<=8]
# build table of professor and sections
Prof=section_info["FirstInstructor"].unique()
professor={}
for i in Prof:
    professor[i]=section_info.loc[section_info.loc[:,"FirstInstructor"]==i,"section"]
print max(section_info["timelength"])
print len(section_info)
# for change the index, do not delete 
section_info.index=range(len(section_info))
# get how many sessions we have in this sechedule
numofsections=sum(section_info["pattern"])
numofprof=len(Prof)

8
514


## Change the size of sample dataset

In [128]:
# select a subset of section id 
# only keep those classes with reg count less than 150, so ll105 can meet the need of seats
section_info=section_info.loc[section_info.loc[:,"Reg Count"]<=149,:]#77
section_info=section_info.reset_index(drop=True)
# randomly choose 100 sections
random.seed(1)
indexofsection=random.sample(section_info.index,85)
section_info=section_info.iloc[indexofsection,:]
section_info=section_info.reset_index(drop=True)
#section_info=section_info.iloc[range(30),:]
#reset dataframe's index 
Prof=section_info["FirstInstructor"].unique()
professor={}
for i in Prof:
    professor[i]=section_info.loc[section_info.loc[:,"FirstInstructor"]==i,"section"]
numofsections=sum(section_info["pattern"])
numofprof=len(Prof)
print numofsections
print numofprof

140
60


In [129]:
# as we just use a 100 sections from origional dataset, we try to manully choose available classroom to make the ratio of 
# # of registed student and # of available seats similiar for origional and new subset data
course20153=course_enrollment.loc[course_enrollment.loc[:,"Term"]==20153,:]
course20153=course20153[(course20153.loc[:,"First End Time"]==course20153.loc[:,"First End Time"])&
                        (course20153.loc[:,"First Begin Time"]==course20153.loc[:,"First Begin Time"])]
course20153=course20153.loc[course20153.loc[:,"Reg Count"]<=149,:]


# calculate timelength for 20153
timelength=[]
for i in course20153.index:
    time1=course20153.loc[i,"First End Time"]
    time2=course20153.loc[i,"First Begin Time"]
    minutediff=((time1.hour*60+time1.minute)-(time2.hour*60+time2.minute)+10)
    halfhour=minutediff/30
    if minutediff%30>0:
        halfhour+=1
    timelength.append(halfhour)
course20153.loc[:,"timelength"]=timelength

# origional ratio of registered student and number of seats
ratio1=float(sum(course20153["Reg Count"]))/(sum(capacity1["Size"]))
print "\n"
print "origional ratio  "+str(ratio1)
# manully choose classroom to make the new ratio is similiar to the origional ratio
capacity=pd.DataFrame({"Room":["JFF LL105","JFF414","ACC201","JKP104","ACC312","BRI5"],
                       "Size":[149,60,48,56,20,42
                              ]})

#capacity=pd.DataFrame({"Room":["JFF LL105"],"Size":[149]})
ratio2=sum(section_info["Reg Count"])/sum(capacity["Size"])
print "\n"
print "origional ratio  "+str(ratio2)



origional ratio  9.69407894737


origional ratio  8.4


## Build variables

In [130]:
# What we need to extract from data
# prediction about student registeration for every section
# how many section we need to schedule
# the pattern, time length of each section
# classroom capacity

# Variables
# course name
C={}
index=0
for i in section_info["section"]:
    C[i]=section_info.loc[section_info.loc[:,"section"]==i,"course"].get(index)
    index+=1
# section of course
I=section_info["section"]
# classroom
J=capacity["Room"]
# pattern of session (less than catagories in origional dataset)
P=["Monday","Tuesday","Wednesday","Thursday","Friday"]
P=pd.Series(i for i in P)
# start time of session (from 8:00am to 9:00pm,so the real classtime is between 8:00am to 10:00pm)
T1={}
time = datetime.timedelta(hours=8,minutes=0, seconds=0)
for i in range(27):
    T1[str(time+timedelta(hours=0.5*i))]=i
T=pd.Series(i for i in range(27))

# students' registeration prediction
student={}
for i in I:
    sum1=sum(course_enrollment.loc[course_enrollment.loc[:,"Section"]==i,"Reg Count"])
    len1=len(course_enrollment.loc[course_enrollment.loc[:,"Section"]==i,"Reg Count"])
    student[i]=round(sum1/len1)
#student=pd.Series(i for i in student)

# seat
seat={}
for index,row in capacity.iterrows():
    seat[row["Room"]]=row["Size"]
#seat=pd.Series(i for i in seat)
    
# pattern of each section
PS={}
index=0
for i in I:
    PS[i]=section_info.loc[section_info.loc[:,"section"]==i,"pattern"].get(index)
    index+=1
#PS=pd.Series(i for i in PS)
    
# timelength of each section
TS={}
index=0
for i in I:
    TS[i]=section_info.loc[section_info.loc[:,"section"]==i,"timelength"].get(index)
    index+=1
#TS=pd.Series(i for i in TS)

#define primetime
primetime=range(4,21)

## Build Model

In [131]:
# Building model
mod=grb.Model()

## Decision Variables

In [132]:
X={}
for i in I: # section id
    for j in J: # classroom
        for p in P: # day of week
             for t in T: # start timeslot of sesstion
                    X[i,j,p,t]=mod.addVar(vtype=grb.GRB.BINARY, name='x[{0},{1},{2},{3}]'.format(i,j,p,t))

                    
# add one more variable PD[f,p,b], f is faculty(professor), p is day of week, b is the building
# PD[f,p,b]=0 means that prof f in p day do not have class in building b 
PD={}
Building=["ACC","BRI","HOH","JFF","JKP"]
for f in Prof:
    for p in P:
        for b in Building:
            PD[f,p,b]=mod.addVar(vtype=grb.GRB.BINARY,name='PD[{0},{1},{2}]'.format(f,p,b))

# add one more numeric variable Dp,f means that professor f have to show up in campus in p day of week
D={}
for f in Prof:
    for p in P:
        D[p,f]=mod.addVar(vtype=grb.GRB.BINARY,name='x[{0}]'.format(p))

#add one more varible PTi. PTi means that how many sessions of section i are in prime time
PT={}
for i in I:
    PT[i]=mod.addVar(vtype=grb.GRB.INTEGER,name='PT[{0}]'.format(i))

# add one more variable y[], which means that whether section i in time t in day p has class in classroom j
# when y is 1 means it has class, when y is 0 means it does not have class
y=mod.addVars(I,J,P,T,vtype=grb.GRB.BINARY)

## Objective

In [133]:
# we assign the highest weight to classroom utilization object, the second id prime time 
# utilization rate, the final one is number of days that professor has class
mod.setObjective((
    # classroom utilization (max)
    3*sum(X[i,j,p,t]*student[i]/seat[j] for i in I for j in J for p in P for t in T)/numofsections
    # number of days that professor has class (min)
    -sum(D[p,f] for f in Prof for p in P)/numofprof
    # number of buildings that professor need to go in anyday that he has class (min
    #-sum(PD[f,p,b] for f in Prof for p in P for b in Building)/numofprof
    # prime time utilization (max)
    +2*sum(30*TS[i]*PT[i] for i in I)/(5*8*60*len(capacity))
),sense=grb.GRB.MAXIMIZE)

## Constraints

In [134]:
# build to series to store the information about one session's section and two session's section
I_twoday=pd.Series()
I_oneday=pd.Series()
for i in I:
    if PS.get(i)==1:
        I_oneday=I_oneday.append(pd.Series(i))
    else:
        I_twoday=I_twoday.append(pd.Series(i))

In [135]:
# constraint1
# in each time slot in each day every professor can only teach one session at the same time
constraint1={}
for t in T:
    for p in P:
        for prof in Prof:
            constraint1[t,p,prof]=mod.addConstr(sum(X[i,j,p,t] for i in professor[prof].values for j in J)<=1)
            

In [136]:
# constraint2
# avoid the conflict between classes
for i in I:
    for j in J:
        for t in T:
            for p in P:
                timeslots=int(TS[i])
                mod.addConstr(y[i,j,p,t]==sum(X[i,j,p,t2] for t2 in range(max(0,t-timeslots+1),t+1)),name="constrain2_"+str(i))

for j in J:
    for t in T:
        for p in P:
            mod.addConstr(sum(y[i,j,p,t] for i in I)<=1)


In [137]:
# this constraint3 is only for those sections whose pattern is 2, which means they have two session every week.
constraint3={}
for i in I_twoday:
    for j in J:
        for t in T:
            constraint3[i,j,"M",t]=mod.addConstr(X[i,j,"Wednesday",t]== X[i,j,"Monday",t],name="") 
            # the section with two sessions must have same 0/1 in M and W
            
            constraint3[i,j,"T",t]=mod.addConstr(X[i,j,"Thursday",t] == X[i,j,"Tuesday",t],name="") 
            # the section with two sessions must have same 0/1 in M and W

In [138]:
# this constraint4 is to control that when we maximum the utilization rate,the number of student registered 
#for one section should less than the classroom capacity
constraint4={}
for i in I:
    for j in J:
        for p in P:
             for t in T:
                    constraint4[i,j,p,t]=mod.addConstr(X[i,j,p,t]*student[i]<=seat[j],name="")
                    

In [139]:
# this constraint5 is to make sure that in all time in all classroom in all day, 
#for each section, sum(X)=required sessions
constraint5={}
for i in I:
    constraint5[i]=mod.addConstr(sum(X[i,j,p,t] for j in J for p in P for t in T)==PS.get(i),name="")    

In [140]:
#constraint6: No Friday class for MW and TH section
constraint6={}
for i in I_twoday:
    constraint6[i] = mod.addConstr(sum(X[i,j,"Friday",t] for j in J for t in T) == 0) 

In [141]:
#constraint7:no class can last longer than the '9PM-9.30PM' block
constraint7={}
for i in I:
    constraint7[i] = mod.addConstr(sum(X[i,j,p,T.max()-ts] for j in J for p in P for ts in range(int(TS[i])-1))<=0)

In [142]:
# building to classroom table
ACC=[]
BRI=[]
HOH=[]
JFF=[]
JKP=[]
for j in J:
    if j[0:3]=="ACC":
        ACC.append(j)
    elif j[0:3]=="BRI":
        BRI.append(j)
    elif j[0:3]=="HOH":
        HOH.append(j)
    elif j[0:3]=="JFF":
        JFF.append(j)
    elif j[0:3]=="JKP":
        JKP.append(j)
BtoC={"ACC":ACC,"BRI":BRI,"HOH":HOH,"JFF":JFF,"JKP":JKP}

In [143]:
# constraint8 for each professor, make them teach in same building for everyday they teach
constraint8={}

for f in Prof:
    for p in P:
        for b in Building:
            courses=professor.get(f)
            classrooms=BtoC.get(b)
            for i in courses:
                for j in classrooms:
                    for t in T:
                        constraint8[f,p,b,i,j,t]=mod.addConstr(PD[f,p,b]>=X[i,j,p,t])
                        
                        
for f in Prof:
    for p in P:
        mod.addConstr(sum(PD[f,p,b] for b in Building)<=1)
        

In [144]:
# constraint9 is to minimum the day that professor show in campus
constraint9={}

for f in Prof:
    for p in P:
        for i in professor.get(f):
            constraint9[f,p,i]=mod.addConstr(sum(X[i,j,p,t] for j in J for t in T)<=D[p,f])    
            

In [145]:
#constraint10 is to maximum the prime time of classrooms used by the course schedule
constraint10={}
for i in I:
    #print i
    constraint10[i]=mod.addConstr(sum(X[i,j,p,t] for j in J for p in P for t in primetime if(t+TS[i])<=20)>=PT[i])


In [146]:
#print "number of constrints "+str(len(constraint10)+len(constraint9)+len(constraint8)+len(constraint7)+len(constraint6)+len(constraint5)+len(constraint4)+len(constraint3)+len(constraint2)+len(constraint1))

## Best solution output

In [147]:
mod.setParam("TimeLimit",600)
mod.optimize()
mod.setParam('OutputFlag',False)

print('Optimal solution:',mod.ObjVal)

Changed value of parameter TimeLimit to 600.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Optimize a model with 234315 rows, 139585 columns and 908220 nonzeros
Variable types: 0 continuous, 139585 integer (139500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+01]
  Objective range  [1e-03, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Presolve removed 202488 rows and 83882 columns (presolve time = 5s) ...
Presolve removed 213532 rows and 95034 columns (presolve time = 10s) ...
Presolve removed 222932 rows and 104434 columns
Presolve time: 14.22s
Presolved: 11383 rows, 35151 columns, 258163 nonzeros
Variable types: 0 continuous, 35151 integer (35151 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Presolve removed 2000 rows and 2000 columns
Presolved: 9383 rows, 33151 columns, 254163 nonzeros

Presolve removed 2064 rows and 2213 columns

Root simplex log...

Iteration    Objective  

In [148]:
# get the small subset of J that have classes in this classroom
J1=[]
for j in J:
    if sum(X[i,j,p,t].x for i in I for p in P for t in T)>0:
        J1.append(j)


In [149]:
# return a blank table
def blanktable():
    T1=[]
    blank=[]
    time = datetime.timedelta(hours=8,minutes=0, seconds=0)
    for i in range(27):
        T1.append(str(time+timedelta(hours=0.5*i)))
        blank.append("")
    schedule=pd.DataFrame({"# timeslot": range(27),
                           "timeslot":T1,
                           "Monday":blank,
                           "Tuesday":blank,
                           "Wednesday":blank,
                           "Thursday":blank,
                           "Friday":blank})
    sched_cols = ['# timeslot','timeslot','Monday','Tuesday','Wednesday','Thursday','Friday']
    schedule = schedule[sched_cols]
    schedule=schedule.set_index('# timeslot')
    return schedule

In [150]:
# put the optimal solution into our beautiful schedule table
timetable={}
for j in J1:
    schedule1=blanktable()
    for i in I:
        for p in P:
            for t in T:
                if X[i,j,p,t].x>0:
                    for a in range(int(TS[i])):
                        schedule1.loc[t+a,p]=C[i]+" / "+str(i)
    timetable["schedule"+j]=schedule1
    

In [151]:
output=pd.ExcelWriter("schedule_sections.xlsx")
for j in J:
    if sum(X[i,j,p,t].x for i in I for p in P for t in T)>0:
        a=j+" "+str(seat[j])
        timetable["schedule"+j].to_excel(output,sheet_name=a,index=False)
output.save()

In [152]:
# make sure that every professor is assigned to one building in the day that they have class
# constraint8
for f in Prof:
    for p in P:
        for b in Building:
            if PD[f,p,b].x>0:
                print f+" "+p+" "+b+" "+str(PD[f,p,b].x)

Louk, Robert, J Monday ACC 1.0
Louk, Robert, J Tuesday ACC 1.0
Louk, Robert, J Wednesday ACC 1.0
Louk, Robert, J Thursday ACC 1.0
Louk, Robert, J Friday ACC 1.0
Wilbur, Therese, Kujawa Monday JFF 1.0
Wilbur, Therese, Kujawa Tuesday BRI 1.0
Wilbur, Therese, Kujawa Wednesday JFF 1.0
Wilbur, Therese, Kujawa Thursday BRI 1.0
Burgos, Miriam, T Monday JFF 1.0
Burgos, Miriam, T Tuesday JFF 1.0
Burgos, Miriam, T Wednesday JFF 1.0
Burgos, Miriam, T Thursday JFF 1.0
Hopkins, Merle, W Monday BRI 1.0
Hopkins, Merle, W Tuesday JFF 1.0
Hopkins, Merle, W Wednesday BRI 1.0
Hopkins, Merle, W Thursday JFF 1.0
Mukherjee, Gourab Monday JFF 1.0
Mukherjee, Gourab Tuesday JFF 1.0
Mukherjee, Gourab Wednesday JFF 1.0
Mukherjee, Gourab Thursday JFF 1.0
Mukherjee, Gourab Friday JKP 1.0
Kovacevich, Rex, Alan Monday BRI 1.0
Kovacevich, Rex, Alan Tuesday ACC 1.0
Kovacevich, Rex, Alan Wednesday ACC 1.0
Kovacevich, Rex, Alan Thursday JKP 1.0
Kovacevich, Rex, Alan Friday BRI 1.0
Ansari, Arif Monday JFF 1.0
Ansari, Ari

In [153]:
# check every professor's class day
# constraint9
for f in Prof:
    for p in P:
        if D[p,f].x>0:
            print f+" "+p+" "+str(D[p,f].x)

Louk, Robert, J Wednesday 1.0
Wilbur, Therese, Kujawa Tuesday 1.0
Wilbur, Therese, Kujawa Thursday 1.0
Burgos, Miriam, T Tuesday 1.0
Burgos, Miriam, T Thursday 1.0
Hopkins, Merle, W Tuesday 1.0
Hopkins, Merle, W Thursday 1.0
Mukherjee, Gourab Friday 1.0
Kovacevich, Rex, Alan Friday 1.0
Ansari, Arif Friday 1.0
Snyder, Kirk, Dylan Tuesday 1.0
Snyder, Kirk, Dylan Thursday 1.0
Kiddoo, Bob Monday 1.0
Kiddoo, Bob Wednesday 1.0
Swenson, Charles, W Monday 1.0
Swenson, Charles, W Wednesday 1.0
Cavanaugh, Lisa, Ann Monday 1.0
Cavanaugh, Lisa, Ann Wednesday 1.0
Salomone, Anthony Friday 1.0
Bristow, Duke Monday 1.0
Bristow, Duke Wednesday 1.0
Bristow, Duke Friday 1.0
Randhawa, Smrity, P Tuesday 1.0
Randhawa, Smrity, P Thursday 1.0
Fast, Nathanael Monday 1.0
Coombs, Michael, Wallace Tuesday 1.0
Coombs, Michael, Wallace Thursday 1.0
Layton, Rose, M Monday 1.0
Layton, Rose, M Wednesday 1.0
Phiroz, Zal Tuesday 1.0
Phiroz, Zal Thursday 1.0
Kim, Peter, Hochin Friday 1.0
Bresnahan, Chris Friday 1.0
Bemis

In [154]:
# check the number of sessions in prime time of every section
# constraint10

for i in I:
    print str(i)+" "+str(PT[i].x)

14279 1.0
16496 0.0
15561 2.0
14522 2.0
14923 1.0
14804 1.0
16271 1.0
15701 0.0
14209 0.0
14061 2.0
16474 2.0
14844 0.0
15560 2.0
14025 2.0
14810 1.0
15398 1.0
14519 2.0
16792 1.0
16671 2.0
14062 0.0
14055 2.0
14907 2.0
16705 1.0
14741 1.0
14495 2.0
14786 2.0
14514 2.0
14828 1.0
14513 2.0
14515 2.0
14497 2.0
14830 1.0
14648 2.0
14057 2.0
16470 0.0
15057 1.0
16298 1.0
14384 2.0
66771 2.0
16504 2.0
14236 2.0
14743 1.0
15372 2.0
16703 1.0
16120 1.0
16321 1.0
14650 2.0
15099 2.0
16549 2.0
16480 2.0
14917 1.0
15108 2.0
14065 0.0
14512 2.0
15723 2.0
14785 0.0
14387 1.0
14968 2.0
15362 2.0
15343 2.0
14725 0.0
14892 1.0
15660 2.0
14920 1.0
14732 0.0
14895 1.0
14096 2.0
66750 2.0
15100 2.0
14381 2.0
14916 1.0
66746 2.0
15592 2.0
14901 0.0
14894 1.0
66781 2.0
15053 1.0
14548 0.0
66769 2.0
14026 0.0
15661 2.0
15904 2.0
16548 0.0
15440 1.0
15753 0.0


In [155]:
mod.presolve()
mod.write('check.lp')

## Result Compare for Report

In [156]:
#sum(professor's teaching days per week)/total number of professors

print "new prof's teaching rate "+str(sum(D[p,f].x for p in P for f in Prof)/len(Prof))
dayofprof=pd.DataFrame(columns=["professor","teaching day"],index=I)
data1=data[["Section","FirstInstructor","FirstDays"]]
num=0
abc=[]
for i in I:
    dayofprof.loc[i,"professor"]=str(data1.loc[data1.loc[:,"Section"]==i,"FirstInstructor"].values)
    dayofprof.loc[i,"teaching day"]=str(data1.loc[data1.loc[:,"Section"]==i,"FirstDays"].values)
numofteachday=0
for j in dayofprof["teaching day"]:
    if len(j)==7:
        numofteachday+=2
    elif len(j)==6:
        numofteachday+=1
print "origional prof's teaching rate "+str(numofteachday/len(Prof))

new prof's teaching rate 1.71666666667
origional prof's teaching rate 2


In [157]:
# method from jonanna
# utilization rate of classroom
# data after optimization
capacity2=capacity1.set_index("Room")
allcapacity=sum(sum(X[i,j,p,t].x for i in I_oneday for p in P for t in T)*capacity2.loc[j,"Size"] for j in J
               )+sum(sum(X[i,j,p,t].x for i in I_twoday for p in P for t in T)*capacity2.loc[j,"Size"] for j in J)/2
allregcount=sum(section_info["Reg Count"])
print allregcount
print allcapacity
print "new classroom utilization rate "+str(allregcount/allcapacity)

# origional data
#data.columns=["Course","Section","FirstDays","FirstBeginTime","FirstEndTime","FirstRoom","FirstInstructor","regcount"]
allcapacity=0
allregcount=0
for i in I:
    #print i
    allregcount=allregcount+int(data.loc[data.loc[:,"Section"]==i,"regcount"].values)
    allcapacity=allcapacity+int(data.loc[data.loc[:,"Section"]==i,"Seats"].values)

print "origional classroom utilization rate "+str(float(allregcount)/allcapacity)

3150.0
5154.0
new classroom utilization rate 0.611175785797
origional classroom utilization rate 0.921157419697


In [158]:
capacity=capacity.set_index("Room")

In [159]:
# new classroom utilization rate based on classroom
one_session=[]
for i in I:
    for j in J:
        if sum(X[i,j,p,t].x for p in P for t in T)>0:
            #print j
            one_session.append(student[i]/capacity.loc[j,"Size"])
            
#print sum(one_session)
#print len(one_session)
print "new classroom utilization rate "+str(float(sum(one_session))/len(one_session))

#utilization rate
#sum(number of registered students)/sum(seats in the assigned classroom)
#print "new classroom utilization rate "+str(sum(section_info["Reg Count"])/sum(capacity["Size"]))

meanreg=[]
capacitysize=0
for j in J:
    cr=data.loc[data.loc[:,"FirstRoom"]==j,"regcount"]
    if len(cr)!=0:
        a=(float(sum(cr))/len(cr))/capacity.loc[j,"Size"]
        #print a
        meanreg.append((float(sum(cr))/len(cr))/capacity.loc[j,"Size"])
        #capacitysize=capacitysize+capacity.loc[j,"Size"]
        #print j
#print sum(meanreg)
#print len(capacity)
print "origional classroom utilization rate "+str(float(sum(meanreg))/len(capacity))



new classroom utilization rate 0.725753717594
origional classroom utilization rate 0.487830203306


In [160]:
#utilization rate prime time of optimized result
#sum(primetime以内的每节课时长[按分钟算])/(教室数*一天8小时*60*5天[按分钟算])
usedprimetime=sum(int(section_info.loc[section_info.loc[:,"section"]==i,"timelength"].values*30*PT[i].x) for i in I)
allprimetime=(8*60*5*len(capacity))
print "new utilization rate of prime time "+str(float(usedprimetime)/allprimetime)

#utilization rate prime time of origional data
#sum(primetime以内的每节课时长[按分钟算])/(教室数*一天8小时*60*5天[按分钟算])
#data[["FirstBeginTime","FirstEndTime"]]
begintime=datetime.time(10,0,0)
endtime=datetime.time(18,0,0)
timedata=data.loc[(data.loc[:,"FirstBeginTime"]>=begintime)&
                  (data.loc[:,"FirstEndTime"]<=endtime),["FirstBeginTime","FirstEndTime"]]
# timelength
timelength=[]
for i in timedata.index:
    time1=timedata.loc[i,"FirstEndTime"]
    time2=timedata.loc[i,"FirstBeginTime"]
    minutediff=((time1.hour*60+time1.minute)-(time2.hour*60+time2.minute)+10)
    halfhour=minutediff/30
    if minutediff%30>0:
        halfhour+=1
    timelength.append(halfhour)
    
usedprimetime=sum(timelength)*30
allprimetime=(8*60*5*len(capacity1))
print "origional utilization rate of prime time "+str(float(usedprimetime)/allprimetime)

new utilization rate of prime time 0.96875
origional utilization rate of prime time 0.342045454545


In [161]:
# save input data
section_info.to_excel("85_sestions.xlsx")

In [162]:
# save input data
capacity=pd.DataFrame({"Room":["JFF LL105","JKP110","JFF414","ACC201","JKP104","ACC312"],
                       "Size":[149,77,60,48,56,20
                              ]})
capacity.to_csv("6_classrooms.csv")

In [164]:
result=pd.DataFrame(columns=["I section","J classroom","P day of week","T time slots"])
num=0
for i in I:
    for j in J:
        for t in T:
            for p in P:
                pass
                #result.loc[num,"I section"]=i
                #result.loc[num,"J classroom"]=j
                #result.loc[num,"P day of week"]=p
                #result.loc[num,"T time slots"]=t
                #num+=1
#result.to_excel("saveX_new_ratio.xlsx")